<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/EfficientNet4_with_normalization_without_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
!pip install tqdm
import rasterio
from rasterio.plot import reshape_as_image
from tqdm import tqdm

!pip install torchgeo
import torchgeo.models

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor

     |████████████████████████████████| 19.3 MB 4.1 MB/s 
     |████████████████████████████████| 204 kB 4.2 MB/s 
     |████████████████████████████████| 431 kB 25.6 MB/s 
     |████████████████████████████████| 409 kB 75.7 MB/s 
     |████████████████████████████████| 584 kB 84.1 MB/s 
     |████████████████████████████████| 1.0 MB 82.2 MB/s 
     |████████████████████████████████| 74 kB 4.4 MB/s 
     |████████████████████████████████| 88 kB 10.8 MB/s 
     |████████████████████████████████| 16.7 MB 104.8 MB/s 
     |████████████████████████████████| 512 kB 80.7 MB/s 
     |████████████████████████████████| 6.3 MB 102.3 MB/s 
     |████████████████████████████████| 112 kB 92.5 MB/s 
     |████████████████████████████████| 596 kB 72.4 MB/s 
     |████████████████████████████████| 136 kB 92.8 MB/s 
     |████████████████████████████████| 1.1 MB 79.6 MB/s 
     |████████████████████████████████| 58 kB 8.6 MB/s 
     |████████████████████████████████| 376 kB 63.8 MB/s 
     |███████████

In [3]:
#local
#Loads tiff folders

import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#folder = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Forest
/content/drive/MyDrive/ML/data/dataset/trainset/HerbaceousVegetation
/content/drive/MyDrive/ML/data/dataset/trainset/Highway
/content/drive/MyDrive/ML/data/dataset/trainset/Industrial
/content/drive/MyDrive/ML/data/dataset/trainset/Pasture
/content/drive/MyDrive/ML/data/dataset/trainset/PermanentCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Residential
/content/drive/MyDrive/ML/data/dataset/trainset/River
/content/drive/MyDrive/ML/data/dataset/trainset/SeaLake


In [4]:
#Define paths

ROOT_PATH = '/content/drive/MyDrive/ML/'
#ROOT_PATH = '/Volumes/GoogleDrive/My Drive/ML'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path_train = '/content/drive/MyDrive/ML/data/dataset/train.csv'
file_path_test = '/content/drive/MyDrive/ML/data/dataset/test.csv'
#file_path_train = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/train.csv'
#file_path_test = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/test.csv'
data_df_train = pd.read_csv(file_path_train)
data_df_test = pd.read_csv(file_path_test)

data_df_train.columns = data_df_train.columns.str.replace('path', 'image_id')
data_df_train.columns = data_df_train.columns.str.replace('Label', 'label')

In [5]:
data_df_train.head()

,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [6]:
type(data_df_train)

pandas.core.frame.DataFrame

In [7]:
#Init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [8]:
#Classes and labels

IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [9]:
#Encoder and decoder

## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

In [10]:
#Open tiff files with rasterio and safe to dictionary

'''import os

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    with rasterio.open(pic, "r") as img:
      arr[i] = np.moveaxis(img.read(), 0, 2)
      i += 1
      
  data[index] = arr'''

'import os\n\ndata = {}\n\nfor i in range(len(subfolders)):\n\n  txtfiles = []\n\n  # traverse whole directory\n  for root, dirs, files in os.walk(subfolders[i]):\n    # select file name\n    for file in files:\n        # check the extension of files\n        if file.endswith(\'.tif\'):\n            txtfiles.append(os.path.join(root, file))\n\n  code = subfolders[i].split(\'/\')[8]\n  index = encode_label(code)\n  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")\n\n  print(code)\n  print(index)\n\n  i = 0\n  for pic in txtfiles:\n    with rasterio.open(pic, "r") as img:\n      arr[i] = np.moveaxis(img.read(), 0, 2)\n      i += 1\n      \n  data[index] = arr'

In [11]:
#Save the files as .npy

'''for i in range(len(data)):
  keys_list = list(data)
  key = keys_list[i]
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)'''

"for i in range(len(data)):\n  keys_list = list(data)\n  key = keys_list[i]\n  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)"

In [12]:
#Reload the .npy files

import os
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#path = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = path.split('/')[8].split('_')[1][5]
  index = int(index)
  data[index] = arr

100%|██████████| 10/10 [01:13<00:00,  7.35s/it]


In [13]:
from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        id = img_id - 1
        img = data[label_encode][id]
        img_rgb = img[:, :, [3,2,1]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [14]:
transform_normalisation = transforms.Compose([
    transforms.ToTensor(),           
    transforms.ConvertImageDtype(torch.float),
])

In [15]:
train_ds = EuroSAT(data_df_train, BASE_PATH, transform_normalisation)
print(len(train_ds))

27000


In [16]:
normalisation_dl = DataLoader(train_ds, batch_size=len(train_ds))

In [17]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [18]:
mean, std = get_mean_and_std(normalisation_dl)

In [19]:
print(mean)
print(std)

tensor([ 946.5516, 1041.8866, 1117.2065])
tensor([594.4782, 395.2131, 333.4205])


In [20]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),              
    #transforms.ConvertImageDtype(torch.float),
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    #transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[946.5544, 1041.8865, 1117.2018],
        std=[594.4802, 395.2151, 333.4477]
    )
])

In [21]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Resize(224),              
    transforms.ConvertImageDtype(torch.float),
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    #transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[943.4290, 578.8717, 628.8640],
        std=[660.1735, 599.4288, 486.5515]
    )
])

In [22]:
train_ds = EuroSAT(data_df_train, BASE_PATH, train_transform)
print(len(train_ds))

27000


In [23]:
train_size = int(0.8 * len(data_df_train))
test_size = len(data_df_train) - train_size
train_ds, validation_ds = torch.utils.data.random_split(train_ds, [train_size, test_size], generator=torch.Generator().manual_seed(seed_value))

In [24]:
validation_ds.dataset.transform = test_transform

In [25]:
train_ds[0]

(tensor([[[-0.4702, -0.4702, -0.4914,  ..., -0.1385, -0.2233, -0.1779],
          [-0.4702, -0.4702, -0.4914,  ..., -0.1385, -0.2233, -0.1779],
          [-0.4414, -0.4414, -0.4687,  ..., -0.2324, -0.1915, -0.1779],
          ...,
          [-0.6808, -0.6808, -0.6747,  ..., -0.7111, -0.7156, -0.7262],
          [-0.6929, -0.6929, -0.6868,  ..., -0.7111, -0.7217, -0.7247],
          [-0.6641, -0.6641, -0.6581,  ..., -0.6914, -0.7126, -0.7429]],
 
         [[ 0.6325,  0.6325,  0.6275,  ...,  0.6508,  0.5774,  0.6041],
          [ 0.6325,  0.6325,  0.6275,  ...,  0.6508,  0.5774,  0.6041],
          [ 0.6241,  0.6241,  0.6308,  ...,  0.6025,  0.6041,  0.6425],
          ...,
          [ 0.5708,  0.5708,  0.5424,  ...,  0.4873,  0.4707,  0.4890],
          [ 0.5441,  0.5441,  0.5240,  ...,  0.5190,  0.4990,  0.4974],
          [ 0.5291,  0.5291,  0.5257,  ...,  0.5441,  0.5391,  0.5024]],
 
         [[ 0.6313,  0.6313,  0.5943,  ...,  0.7607,  0.7196,  0.7484],
          [ 0.6313,  0.6313,

In [26]:
print(len(train_ds))
print(len(validation_ds))

21600
5400


In [27]:
model = models.efficientnet_b4(pretrained=True)

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth


  0%|          | 0.00/74.5M [00:00<?, ?B/s]

Model

In [28]:
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [29]:
pretrainedBool = True
if pretrainedBool:
    for param in model.parameters():
        param.requires_grad = False
model.classifier = nn.Sequential(
                      nn.Linear(1792, 256),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(256, NUM_CLASSES),
                        )

In [30]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cuda computation enabled


In [31]:
model = model.to(device)

In [32]:
# define the optimization criterion / loss function
ce_loss = nn.CrossEntropyLoss()

In [33]:
ce_loss = ce_loss.to(device)

In [34]:
config={
"epochs": 20,
"mini_batch_size": 32,
"learning_rate": 0.01,
"opt_func": torch.optim.Adam,
"milestones": [4,8,10],
"weight_decay": 0,
"description": "resnetb4_without_softmax_with_224_transform"
}

In [35]:
train_dl = DataLoader(train_ds, batch_size=config.get("mini_batch_size"), shuffle=True)

In [313]:
#Empty cache
torch.cuda.empty_cache()

In [ ]:
!pip install wandb
!wandb login
import wandb
#Key: 9da448bfaa162b572403e1551114a17058f249d0

In [314]:
run = wandb.init(project="ml-challenge", entity="nicikess", config=config)

loss,▆█▅▅▃▅▄▁▇▇█▂▄▃▃▄▄▁▃▃▅▂▂▃▃▂▂▂▃▁▃▂▃▂▄▂▃▂▂▃
loss,0.70476


In [317]:
def train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay):
  
  # init collection of training epoch losses
  train_epoch_losses = []

  # set the model in training mode
  model.train()

  optimizer = opt_func(model.classifier.parameters(),lr=learning_rate, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

  # train the CIFAR10 model
  for epoch in range(epochs):
      
      # init collection of mini-batch losses
      train_mini_batch_losses = []

      # iterate over all-mini batches
      for i, (images, labels) in enumerate(train_dl):

          #print(images)

          # push mini-batch data to computation device
          images = images.to(device)
          labels = labels.to(device)

          # run forward pass through the network
          output = model(images)

          # reset graph gradients
          optimizer.zero_grad()
          
          # determine classification loss
          loss = ce_loss(output, labels)

          wandb.log({"loss": loss})
          
          # run backward pass
          loss.backward()
          
          # update network paramaters
          optimizer.step()
          
          # collect mini-batch reconstruction loss
          train_mini_batch_losses.append(loss.data.item())

      # determine mean min-batch loss of epoch
      train_epoch_loss = np.mean(train_mini_batch_losses)

      print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

      scheduler.step()

      print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

      #Added
      #result = evaluate(model, vali_dataloader)

      # print epoch loss
      now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
      print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
      
      #if (epoch%10 == 0): 
          # set filename of actual model
      model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))
          # save current model to GDrive models directory
      torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))

      # determine mean min-batch loss of epoch
      train_epoch_losses.append(train_epoch_loss)

In [320]:
epochs = config.get("epochs")
learning_rate = config.get("learning_rate")
weight_decay = config.get("weight_decay")
opt_func = config.get("opt_func")
milestones = config.get("milestones")
max_learning_rate = config.get("max_learning_rate")

train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay)

Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f8acfb2bb10>> (for post_run_cell):


Exception: ignored

Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 152, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 138, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 405, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 226, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/usr/local/lib/python

In [44]:
resnet50_vali_dataloader = DataLoader(validation_ds, batch_size=5400)

In [45]:
# Display image and label.
validation_features, validation_labels = next(iter(resnet50_vali_dataloader))
print(f"Labels batch shape: {validation_labels.size()}")

Labels batch shape: torch.Size([5400])


In [36]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_5.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [37]:
# set model in evaluation mode
best_model.eval()

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [38]:
predictionsVal = list()
from PIL import Image
from numpy import vstack
for i in tqdm(range(len(validation_ds))):
  input = validation_ds[i]
  input = input[0].unsqueeze(0)
  #print(input)
  #input = test_transform(input)
  #x = x[None, :]
  input = input.to(device)
  #print(type(input))
  yhat = best_model(input)
  # retrieve numpy array
  yhat = yhat.detach()
  # convert to class labels
  yhat = torch.argmax(yhat, axis=1)
  # reshape for stacking
  yhat = yhat.reshape((len(yhat), 1))
  # store
  predictionsVal.append(yhat.cpu())
predictionsVal = vstack(predictionsVal)
predictionsVal = torch.tensor(predictionsVal)

100%|██████████| 5400/5400 [01:58<00:00, 45.72it/s]


In [39]:
np.shape(validation_ds[3][0])

torch.Size([3, 64, 64])

In [40]:
list_tens = list()

for i in range(len(predictionsVal)):
  list_tens.append(decode_target(predictionsVal[i].item()))

In [46]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['HerbaceousVegetation', 'PermanentCrop', 'Forest', 'Industrial', 'Highway', 'River', 'Residential', 'SeaLake', 'AnnualCrop', 'Pasture'])
dict_values([959, 442, 3158, 63, 326, 108, 258, 55, 22, 9])


In [47]:
from sklearn import metrics
acc = metrics.accuracy_score(validation_labels.detach().cpu(), predictionsVal.detach().cpu())
print(acc)

0.12796296296296297


In [ ]:
wandb.log({"acc": acc})

In [ ]:
#Finish run
run.finish()

In [54]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/testset/'
npyfiles = []
datatest = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = int(path.split('_')[1].split('.')[0])
  datatest[index] = arr

100%|██████████| 4232/4232 [01:25<00:00, 49.63it/s] 


In [51]:
from pathlib import Path
class EuroSATTest(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        img = datatest[idx]
        img = img[:, :, [4,3,2]]
        img = np.float32(img)
        if self.transform:
            img = self.transform(img)
        return img

In [52]:
def get_mean_and_std_test(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [55]:
test_ds = EuroSATTest(data_df_test, BASE_PATH, transform_normalisation)
normalisation_dl = DataLoader(test_ds, batch_size=len(test_ds))
mean, std = get_mean_and_std_test(normalisation_dl)

In [ ]:
print(std)

tensor([660.1735, 599.4288, 486.5515])


In [56]:
#Test data
test_ds = EuroSATTest(data_df_test, BASE_PATH, test_transform)
print(len(test_ds))

4232


In [ ]:
test_ds[0]

In [57]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_5.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [58]:
# set model in evaluation mode
best_model.eval()

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(

In [59]:
predictions = list()
from PIL import Image
from numpy import vstack
for i in tqdm(range(len(test_ds))):
  input = test_ds[i]
  input = input.unsqueeze(0)
  #print(input)
  #input = test_transform(input)
  #x = x[None, :]
  input = input.to(device)
  #print(type(input))
  yhat = best_model(input)
  # retrieve numpy array
  yhat = yhat.detach()
  # convert to class labels
  yhat = torch.argmax(yhat, axis=1)
  # reshape for stacking
  yhat = yhat.reshape((len(yhat), 1))
  # store
  predictions.append(yhat.cpu())
predictions = vstack(predictions)
predictions = torch.tensor(predictions)

100%|██████████| 4232/4232 [01:29<00:00, 47.19it/s]


In [62]:
list_tens = list()

for i in range(len(predictions)):
  list_tens.append(decode_target(predictions[i].item()))

In [63]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['Forest', 'HerbaceousVegetation', 'Industrial', 'Residential', 'AnnualCrop', 'PermanentCrop', 'SeaLake', 'Highway', 'River', 'Pasture'])
dict_values([2518, 1014, 42, 226, 11, 210, 38, 128, 39, 6])


In [ ]:
from sklearn import metrics
metrics.accuracy_score(validation_labels.detach().cpu(), predictions.detach().cpu())

0.8346296296296296

In [297]:
test_indices_path = '/content/drive/MyDrive/ML/data/dataset/test.csv'

final_submission_file = pd.read_csv(test_indices_path)
prediction_list = []

# turning numeric predictions to strings (as in kaggle example submission)
for p in predictions:
  prediction_list.append(decode_target(p.item()))

final_submission_file["label"] = prediction_list
### source with alternatives methods for appending to csv should this fail: https://stackoverflow.com/questions/26666919/add-column-in-dataframe-from-list
final_submission_file.to_csv('/content/drive/MyDrive/ML/data/dataset/final_submission_ResNet18.csv', index=None)